In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

from statsmodels.nonparametric.smoothers_lowess import lowess

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sb
import pymannkendall as mk
from scipy.fft import fft, fftfreq
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.signal import correlate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

import icclim
import datetime
import xarray as xr
import warnings
warnings.filterwarnings('ignore')

### new global

In [2]:
#new_global = pd.read_csv('Complete_TAVG_summary.txt', skiprows=20, delimiter='\t')
new_global = pd.read_excel('new_global.xlsx')

In [3]:
new_global.head(3)

,Year,Annual Anomaly,Annual Unc.,Five-year Anomaly,Five-year Unc.
0,1750,-1.420,0.883,NaN,NaN
1,1751,-1.515,1.161,NaN,NaN
2,1753,-1.175,0.915,NaN,NaN


In [4]:
ox = pd.read_excel('ox.xlsx', skiprows=4, skipfooter=8)
ox

# skip 2020: no data

,Unnamed: 0,Jan,Feb,Mar,Apr,May,June,July,Aug,Sept,Oct,Nov,Dec,Annual,Unnamed: 14,YEAR,Winter,Spring,Summer,Autumn
0,1811,#,#,#,#,#,#,#,#,#,#,#,#,NaN,NaN,1811,NaN,NaN,NaN,NaN
1,1812,#,#,#,#,#,#,#,#,#,#,#,#,NaN,NaN,1812,NaN,NaN,NaN,NaN
2,1813,#,#,#,#,#,#,#,#,#,#,#,2.888889,NaN,NaN,1813,NaN,NaN,NaN,NaN
3,1814,-2.578938,1.714198,3.186579,10.561728,10.330944,13.019441,17.08184,15.848865,13.203704,8.828554,5.358025,5.494624,8.504130,NaN,1814,0.674716,8.026417,15.316715,9.130094
4,1815,-1.482677,4.502714,5.106355,7.513867,13.772581,15.361667,15.832258,16.382258,13.731367,8.144839,0.845833,0.941226,8.408932,NaN,1815,2.838220,8.797601,15.858728,7.574013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2015,5.020968,4.25,7.325806,10.186667,12.245161,15.636667,17.530645,17.135484,13.613333,11.493548,10.236667,10.8,11.335068,NaN,2015,4.935484,9.919211,16.767599,11.781183
205,2016,6.195161,5.898276,6.18871,8.485,13.51129,15.99,18.498387,18.525806,16.968333,11.191935,6.363333,6.270968,11.194262,NaN,2016,7.631146,9.395000,17.671398,11.507867
206,2017,4.009677,6.691071,9.717742,9.903333,14.309677,17.76,18.609677,16.995161,14.58,13.058065,7.075,5.414516,11.540959,NaN,2017,5.657239,11.310251,17.788280,11.571022
207,2018,6.030484,3.341071,5.814516,10.846667,14.314516,17.208333,21.048387,18.379032,15.245,11.470968,8.61,7.41129,11.696973,NaN,2018,4.928690,10.325233,18.878584,11.775323


In [5]:
ox = ox.iloc[:, :-7].copy()
cols = ox.columns.to_list()
cols[0] = 'Year'
ox.columns = cols

In [6]:
ox = ox.iloc[2:].copy()

In [7]:
ox.set_index('Year', drop=True, inplace=True)

In [8]:
ox.head(3)

,Jan,Feb,Mar,Apr,May,June,July,Aug,Sept,Oct,Nov,Dec
Year,,,,,,,,,,,,
1813,#,#,#,#,#,#,#,#,#,#,#,2.888889
1814,-2.578938,1.714198,3.186579,10.561728,10.330944,13.019441,17.08184,15.848865,13.203704,8.828554,5.358025,5.494624
1815,-1.482677,4.502714,5.106355,7.513867,13.772581,15.361667,15.832258,16.382258,13.731367,8.144839,0.845833,0.941226


In [9]:
ox_stack = ox.stack().to_frame(name='ox')
ox_dates = pd.to_datetime(
ox_stack.index.get_level_values(level=1).astype(str) + '-' + ox_stack.index.get_level_values(level=0).astype(str), format='mixed')
ox_stack.index = ox_dates

In [10]:
ox_stack['ox'].replace('#', np.nan, inplace=True)

In [11]:
px.line(ox_stack.resample('1Y').mean())

In [12]:
# mean of 1850-1900
mean_1850_1900 = ox_stack.loc[pd.to_datetime('1850-01-01'):pd.to_datetime('1900-12-12')].mean()
mean_1850_1900

ox    9.602943
dtype: float64

In [13]:
ox_stack_1Y = ox_stack.resample('1Y').mean()

In [14]:
ox_stack_annual_change = ox_stack_1Y - ox_stack_1Y.shift(1)

In [15]:
# G(t) means it's a function of time, so a continuous data series

global_temp = pd.read_csv('1850-2023.csv', skiprows=4)
global_temp

,Year,Anomaly
0,185001,-0.37
1,185002,-0.13
2,185003,-0.24
3,185004,-0.20
4,185005,-0.19
...,...,...
2082,202307,1.13
2083,202308,1.23
2084,202309,1.43
2085,202310,1.36


In [16]:
global_temp['Year'] = global_temp['Year'].astype(str)
global_temp['Year'] = pd.to_datetime(
                        global_temp['Year'].str.slice(start=4) + '-' + 
                        global_temp['Year'].str.slice(stop=4))
global_temp.set_index('Year', drop=True, inplace=True)

In [17]:
px.scatter(global_temp)

In [18]:
# only keep the dates from global where we have the oxford data: left join
ox_stack_global = ox_stack.merge(global_temp, left_index=True, right_index=True, how='left').dropna(how='all')

In [19]:
ox_stack_global_1Y = ox_stack_global.resample('1Y').mean()

In [20]:
ox_stack_global_1Y['ox_change'] = ox_stack_global_1Y['ox'] - ox_stack_global_1Y['ox'].shift(1)

In [21]:
new_global.index = pd.to_datetime(new_global['Year'].astype(str) + '-12-31')
ox_stack_global_1Y = ox_stack_global_1Y.merge(new_global, left_index=True, right_index=True, how='left')
ox_stack_global_1Y['Anomaly'] = ox_stack_global_1Y['Annual Anomaly']

In [22]:
ox_stack_global_1Y.drop(columns=[
    'Annual Anomaly','Annual Unc.',	'Five-year Anomaly', 'Five-year Unc.', 'Year'],
    inplace=True)

In [23]:
ox_stack_global_1Y.tail(3)

,ox,Anomaly,ox_change
2017-12-31,11.510327,1.304,0.336393
2018-12-31,11.643355,1.142,0.133029
2019-12-31,12.331190,1.340,0.687835


In [24]:
px.line(ox_stack_global_1Y)

# Anomaly is 1.14 - ok-ish from the raw data
# the smoothed value is not really smoothed, so play around with the values. 
# need to make the line follow the the raw data better
# then smoothed 2018 data should roughly be around 1.2 

In [25]:
# may need to redo/ignore ox_change
# 1. 41-year lowess filter on ox
# 2. same for Anomaly (global)
# 3. local = m * smoothed global + c
# correlation ox and GMST: 0.45
# correlation between ox and smoothed GMST: 0.67
# correlation between both smoothed: 0.98

In [26]:
fit_ox_anomaly = px.scatter(ox_stack_global_1Y, x='Anomaly', y='ox', trendline='ols')
fit_ox_anomaly.show()

In [27]:
#px.get_trendline_results(fit_ox_anomaly).values[0][0].summary() # great p-value

In [28]:
#ox_dt = ox_lowess[:, 0].astype('datetime64[ns]')
#ox_dt = pd.to_datetime(ox_dt)

In [29]:
it, frac = 41, 0.25

# change number_of_nans to datetime merge later

ox_lowess = lowess(endog=ox_stack_global_1Y['ox'], 
            exog=ox_stack_global_1Y.index,
            it=it,
            frac=frac
            )

global_lowess = lowess(endog=ox_stack_global_1Y['Anomaly'],
                exog=ox_stack_global_1Y.index,
                it=it,
                frac=frac
                )

ox_lowess_df = pd.DataFrame(ox_lowess)
ox_lowess_df[0] = pd.to_datetime(ox_lowess_df[0].astype('datetime64[ns]') )
ox_lowess_df.columns = ['date', 'ox smoothed']
#ox_lowess_df.head(3)

global_lowess_df = pd.DataFrame(global_lowess)
global_lowess_df[0] = pd.to_datetime(global_lowess_df[0].astype('datetime64[ns]') )
global_lowess_df.columns = ['date', 'Anomaly smoothed']
#global_lowess_df.head(3)


In [30]:
ox_stack_global_1Y = ox_stack_global_1Y.merge(ox_lowess_df, 
                                            left_index=True, right_on='date', how='left' )

ox_stack_global_1Y = ox_stack_global_1Y.merge(global_lowess_df, 
                                            left_on='date', right_on='date', how='left')
                                            
ox_stack_global_1Y.set_index('date', inplace=True)

In [31]:
ox_stack_global_1Y.head(3)

,ox,Anomaly,ox_change,ox smoothed,Anomaly smoothed
date,,,,,
1813-12-31,2.888889,-0.948,NaN,9.226528,-0.923275
1814-12-31,8.504130,-1.041,5.615241,9.234122,-0.919776
1815-12-31,8.387691,-1.474,-0.116440,9.241334,-0.916308


In [32]:
#pd.Series(global_lowess[:, 0]) # 170 rows, matches Anomaly after dropna. without dropna = 207

In [33]:
# s: 1.19k, n=0.1k, snr=11.3
# # s: 1.73k, n=0.54k, snr=3.2
 
# visualize data, not fitting
# one raw irish, smoothed irish, global raw, smoothed global

px.scatter(
        #x=ox_lowess[:, 0], 
        #x=ox_stack_global_1Y.index,
        #y=[ox_lowess[:, 1], ox_stack_global_1Y['ox']],
        #y=[ox_stack_global_1Y['ox'],  ox_stack_global_1Y['Anomaly'] ]
        #trendline='ols'

        ox_stack_global_1Y,
        y=['ox', 'Anomaly', 'ox smoothed', 'Anomaly smoothed']
    )

In [34]:
# fitting
# WANT THIS ONE, X= GLOBAL SMOOTHED, Y=IRISH SMOOTHED

linear_fit = px.scatter(ox_stack_global_1Y,
                    x='Anomaly smoothed', y='ox smoothed', 
                    #x='Anomaly', y='ox',
                    trendline='ols')

params = px.get_trendline_results(linear_fit).iloc[0].values[0]
alpha, intercept = params.params[1], params.params[0]

ox_stack_global_1Y['ox calculated'] = alpha * ox_stack_global_1Y['Anomaly smoothed'] + intercept

print(np.array([alpha, intercept, params.rsquared]).round(2))

linear_fit.show()

[ 1.01 10.13  0.97]


In [35]:
px.line(ox_stack_global_1Y.loc[pd.to_datetime('1850-01-01'):],
            x=ox_stack_global_1Y.loc[pd.to_datetime('1850-01-01'):].index, 
            y=['ox', 'ox calculated']
            )

In [36]:
#px.get_trendline_results(linear_fit).iloc[0].values[1].summary()

In [37]:
#fit = px.get_trendline_results(linear_fit).iloc[0].values[1]
#alpha, intercept = fit.params[1], fit.params[0]
#alpha, intercept

In [38]:
#px.line(ox_stack_global_1Y, y='Anomaly')

## SNR

In [39]:
ox_stack_global_1Y.tail(4)

,ox,Anomaly,ox_change,ox smoothed,Anomaly smoothed,ox calculated
date,,,,,,
2016-12-31,11.173933,1.431,-0.115645,11.503416,1.196501,11.329477
2017-12-31,11.510327,1.304,0.336393,11.538557,1.225372,11.358536
2018-12-31,11.643355,1.142,0.133029,11.573554,1.254242,11.387594
2019-12-31,12.331190,1.340,0.687835,11.608441,1.283112,11.416651


In [40]:
#ox_stack_global_1Y['ox_change_from_start'] = ox_stack_global_1Y['ox'] - ox_stack_global_1Y['ox'].iloc[0]

In [41]:
px.line(ox_stack_global_1Y, 
       markers='x'
        #y=['ox', 'ox smoothed', 'ox_change']
        )

In [42]:
ox_stack_global_1Y['residual'] = (
                                (alpha * ox_stack_global_1Y['Anomaly smoothed'] + intercept) -
                                   ox_stack_global_1Y['ox']
                                   )
px.bar(ox_stack_global_1Y, 
             x=ox_stack_global_1Y.index,
             y='residual')

In [43]:
# L(t) = alpha * G(t) + beta

ox_std = ((alpha * ox_stack_global_1Y['Anomaly smoothed'] + intercept) - ox_stack_global_1Y['ox']).std()
ox_std
# maybe this is the right one to use. map the values back to the raw data, not the smoothed one

#((alpha * ox_stack_global_1Y['Anomaly smoothed'] + intercept) - ox_stack_global_1Y['ox smoothed']).std()

# the formula and the OLS formula match ,the value is being calculated correctly from interpretation
# but why is it so different than the paper's?

0.729148465269228

In [44]:
# create figure 1
fig = px.line(ox_stack_global_1Y,
        x=ox_stack_global_1Y.index,
        y=['ox', 'ox smoothed']
        )

fig.add_hrect(y0=9.18, y1=10.62, line_width=0, fillcolor='pink', opacity=.6) # instead of hardcoding the y0 and y1 values, use the formula below
fig.add_hrect(y0=8.46, y1=11.34, line_width=0, fillcolor='red', opacity=.3)

# SNR: 1.73
# mean: 9.9 # ox_stack_global_1Y['ox'].mean()
# 1*std: 0.72 # ox_std
# 2*std: 1.44 # ox_std * 2

# add horizontal lines at 9.9 +/- 0.72  (one colour) -- this formula for pink
# 9.9 +/- 1.44 (different colour) -- for red
# this is SNR chart like figure 1. 


| value | calculation | paper |
| --- | ---- | --- |
| alpha | 1.007 | 1.45 |
| G2018 | 1.25 | 1.19 | 
| signal: alpha*G | 1.26 | 1.72 |
| noise | 0.09 (smoothed), 0.72 (actual values) | 0.54 |
| SNR | 14.0, 1.73 (actual) | 3.2 |

* assuming this is the correct value, your SNR is just 1.26/0.09 -- too high


In [45]:
# signal global: G2018 -- 1.19 K
# signal local: alpha * G
# noise: stdev of residuals L-alpha*G

In [46]:
alpha, intercept # alpha in the paper is 1.45 +/- .25 (95 % CI), not too bad 1.2-1.7

# G2018 from the paper: 1.19 K --x 1.25

(1.0065044821539026, 10.12519313560093)

In [47]:
1.73/1.19

1.453781512605042

In [48]:
# signal of global
alpha * 1.25, alpha * 1.26 # G2018: paper signal is 1.19, we have 1.26

(1.2581306026923782, 1.2681956475139173)

In [49]:
# SNR = signal / noise


In [50]:
ox_stack_global_1Y['signal local'] = alpha * ox_stack_global_1Y['Anomaly smoothed']

In [51]:
np.round((ox_stack_global_1Y['Anomaly smoothed'] - ox_stack_global_1Y['signal local']).std(), 5)

0.00355

In [52]:
ox_stack_global_1Y

,ox,Anomaly,ox_change,ox smoothed,Anomaly smoothed,ox calculated,residual,signal local
date,,,,,,,,
1813-12-31,2.888889,-0.948,NaN,9.226528,-0.923275,9.195913,6.307024,-0.929281
1814-12-31,8.504130,-1.041,5.615241,9.234122,-0.919776,9.199434,0.695304,-0.925759
1815-12-31,8.387691,-1.474,-0.116440,9.241334,-0.916308,9.202925,0.815235,-0.922268
1816-12-31,7.785517,-1.816,-0.602173,9.248198,-0.912848,9.206407,1.420890,-0.918786
1817-12-31,9.071523,-1.553,1.286006,9.254722,-0.909401,9.209877,0.138354,-0.915316
...,...,...,...,...,...,...,...,...
2015-12-31,11.289579,1.237,-0.472611,11.467999,1.167550,11.300337,0.010758,1.175144
2016-12-31,11.173933,1.431,-0.115645,11.503416,1.196501,11.329477,0.155544,1.204284
2017-12-31,11.510327,1.304,0.336393,11.538557,1.225372,11.358536,-0.151791,1.233343


In [53]:
# # linear = [
# #     df_irish.resample('1Y').mean().drop(columns=['irrain mean']), 
# #     df_global[['temp c ffill', 'modern temp c']].mean(1).to_frame(name='temp all c').iloc[-500:].dropna()
# # ]

# # linear[0]['year reset'] = linear[0]['year reset'].astype(int)
# # linear[1].index = linear[1].index.astype(int)


# # linear = linear[0].merge(linear[1], left_on='year reset', 
# #                          right_index=True, how='outer')#.dropna().to_excel('linear.xlsx', index=True)
# # linear.head(10)

# linear = [
#     df_irish.resample('1Y').mean().drop(columns=['irrain mean']), 
#     df_global[['temp c ffill', 'modern temp c']].mean(1).to_frame(name='temp all c').iloc[-500:].dropna(),
#     df_global[['anom modern anom']]
# ]
# linear_0_1 = linear[0].merge(linear[1], 
#                              left_on='year reset', right_index=True, 
#                              how='outer').sort_values('year reset')

# linear_0_1.reset_index(drop=False, inplace=True)

# #linear_0_2 = linear_0_1.merge(linear[2], left_on='year reset', right_index=True, how='outer')
# linear_0_2 = linear_0_1.merge(linear[2], 
#                  left_on='year reset', right_index=True,
#                   how='outer').sort_values('year reset')

# linear_0_2.reset_index(drop=False, inplace=True)
# linear_0_2.set_index('year reset', drop=True, inplace=True)
# linear_0_2.to_excel('linear_0_2x.xlsx')
# linear_0_2.tail()
# linear_0_2[['index', 'irtemp mean', 'temp all c', 'anom modern anom']].ffill().dropna(subset='index')

#####################################################

# linear[0]['year reset'] = linear[0]['year reset'].astype(int)
# linear[1].index = linear[1].index.astype(int)
# linear[2].index = linear[2].index.astype(int)

# linear01 = linear[0].merge(linear[1], left_on='year reset', 
#                          right_index=True, how='outer')#.dropna().to_excel('linear.xlsx', index=True)
# linear = linear01.merge(linear[2], left_on='year reset', 
#                          right_index=True, how='outer')#.dropna().to_excel('linear.xlsx', index=True)
# linear.head(10)
# linear_0_2['irtemp mean k'] = linear_0_2['irtemp mean']+273.15
# linear_0_2.tail(3)
# linear_0_2['irtemp mean k gmst'] = linear_0_2['irtemp mean']-1.24
# linear_0_2.tail(3)
# linear_0_2['irtemp mean gmst c'] = linear_0_2['irtemp mean k gmst']-273.15
# linear_0_2.tail(3)
# linear_0_2['iranomc'] = linear_0_2['irtemp mean'] - linear_0_2['irtemp mean gmst c']
# linear_0_2.tail(5)

# linear_0_2['irtemp meang'] = linear_0_2['irtemp mean'] - 1.24
# linear_0_2.tail(5)
# linear_0_2.reset_index()
# linear_0_2.to_excel('linear_02 0501.xlsx', index=False)
# linear_0_2.reset_index().plot(x='index', y=['irtemp mean'], kind='scatter')
# # linear_0_2.dropna().plot(x='dmy float', y=['irtemp mean', 'temp all c'], figsize=(15,5),
# #             secondary_y='temp all c');

# linear_0_2.plot(x='dmy float', y=['irtemp mean', 'temp all c'], figsize=(15,5),
#             secondary_y='temp all c');

# # linear_0_2.plot(x='dmy float', y=['anom modern anom', 'irtemp gsmt'], figsize=(15,5),
# #             secondary_y='temp all c'
# #             );

# fit an OLS with px.scatter
# get the slope and intercept: L(t) = alpha * G(t) + β
# signal is alpha * G
# noise: component is defined as the standard deviation of the residuals (L − ∝G)
# SNR: np.where

# linear_0_2.iloc[10:]
# linear_0_2[['temp all c', 'irtemp gsmt']]

#linear_0_2.dropna(inplace=True)
# linear_fit = px.scatter(linear.iloc[6:], x='temp all c', y='irtemp mean', trendline='ols')
# linear_fit.show()

# linear_fit = px.scatter(linear.iloc[6:], x='iranomc', y='irtemp mean', trendline='ols')
# linear_fit.show()

# linear_fit = px.scatter(linear_0_2.iloc[6:], x='temp all c', y='irtemp gsmt', trendline='ols')
# linear_fit.show()
# fit = px.get_trendline_results(linear_fit).iloc[0].values[0]
# alpha, intercept = fit.params[1], fit.params[0]
# alpha, intercept

# fit = px.get_trendline_results(linear_fit).iloc[0].values[0]
# alpha, intercept = fit.params[1], fit.params[0]
# alpha, intercept
# 1.24K  = -271.91 C

# linear['signal'] = alpha * linear['temp all c']
# linear['fitted'] = alpha * linear['temp all c'] + intercept
# noise = (linear['fitted'] - linear['signal']).std()
# linear['SNR'] = linear['signal'] / noise

# linear['signal'] = alpha * linear['iranomc']
# linear['fitted'] = alpha * linear['iranomc'] + intercept
# noise = (linear['fitted'] - linear['signal']).std()
# linear['SNR'] = linear['signal'] / noise
# linear
# linear['SNR'].describe()
# np.where(linear['SNR'] < 1, 'normal', np.nan)
# linear['SNR label'] = np.where(linear['SNR'] < 1, 'normal', np.nan)
# linear['SNR label'] = np.where((linear['SNR']>=1) & (linear['SNR'] < 2), 'unusual', linear['SNR label'])
# linear['SNR label'] = np.where((linear['SNR']>=2) & (linear['SNR'] < 3), 'unfamiliar', linear['SNR label'])
# linear['SNR label'] = np.where(linear['SNR'] >= 3, 'unknown')

# linear['SNR label'] = np.where((normal, 'normal'), (unusual, 'unusual'), (unfamiliar, 'unfamiliar'), (unknown, 'unknown'))


In [54]:
# linear['signal'] = alpha * linear['temp all c']
# linear['fitted'] = alpha * linear['temp all c'] + intercept
# noise = (linear['fitted'] - linear['signal']).std()
# linear['SNR'] = linear['signal'] / noise


# linear['signal'] = alpha * linear['']
# linear['fitted'] = alpha * linear[''] + intercept
# noise = (linear['fitted'] - linear['signal']).std()
# linear['SNR'] = linear['signal'] / noise
# linear['SNR']
# relative to the mean of 1850–1900
# ox: α = 1.45 ± 0.25
# “signal” for Oxford is αG2018 = 1.72 ± 0.30 K
# “noise,” = 0.54K
# S/N ratio of 3.2 ± 0.5 # from hawkins paper

# fit an OLS with px.scatter
# get the slope and intercept: L(t) = alpha * G(t) + β
# signal is alpha * G
# noise: component is defined as the standard deviation of the residuals (L − ∝G)
# SNR: np.where

# linear['SNR label'] = np.where(linear['SNR'] < 1, 'normal', np.nan)
# linear['SNR label'] = np.where((linear['SNR']>=1) & (linear['SNR'] < 2), 'unusual', linear['SNR label'])
# linear['SNR label'] = np.where((linear['SNR']>=2) & (linear['SNR'] < 3), 'unfamiliar', linear['SNR label'])
# linear['SNR label'] = np.where(linear['SNR'] >= 3, 'unknown')

# linear['SNR label'] = np.where((normal, 'normal'), (unusual, 'unusual'), (unfamiliar, 'unfamiliar'), (unknown, 'unknown'))

# smoothing
# ox_stack_global_1Y['ox smoothed'] = ox_lowess[:, 1]


# number_of_nans = ox_stack_global_1Y['Anomaly'].isna().value_counts().loc[True]

# # always assume all the nans are the top of the table

# ox_stack_global_1Y['Anomaly smoothed'] = np.concatenate([
#                                             [np.nan] * number_of_nans,
#                                             global_lowess[:, 1]
#                                         ])

In [55]:
df_irtemp_1834_1958= pd.read_csv('test.csv')
df_irtemp_1834_1958.head(3)

,datetime,irtemp c,min c,max c
0,1834-01-01,6.95,3.9,5.0
1,1834-02-01,5.50,4.4,3.3
2,1834-03-01,16.10,10.0,11.1


In [56]:
df_irtemp_1834_1958['datetime'] = pd.to_datetime(df_irtemp_1834_1958['datetime'])
#df = df.set_index('datetime')
# ds = xr.Dataset.from_dataframe(df)
df_irtemp_1834_1958

,datetime,irtemp c,min c,max c
0,1834-01-01,6.95,3.9,5.0
1,1834-02-01,5.50,4.4,3.3
2,1834-03-01,16.10,10.0,11.1
3,1834-04-01,14.45,8.9,10.0
4,1834-05-01,14.70,9.4,10.0
...,...,...,...,...
45650,1958-12-27,15.35,6.7,12.0
45651,1958-12-28,15.40,5.4,12.7
45652,1958-12-29,10.70,3.6,8.9
45653,1958-12-30,11.05,4.7,8.7


In [57]:

# cold_spell = (df_irtemp_1834_1958['min c'].rolling(window=6).min() < np.percentile(df_irtemp_1834_1958['min c'], 10))
# warm_spell = (df_irtemp_1834_1958['max c'].rolling(window=6).max() > np.percentile(df_irtemp_1834_1958['max c'], 90))
# frost_days = (df_irtemp_1834_1958['min c'] < 0)
# growseas_length = ((df_irtemp_1834_1958['irtemp c'] > 5) & (df_irtemp_1834_1958['datetime'].dt.month < 7)) | ((df_irtemp_1834_1958['irtemp c'] < 5) & (df_irtemp_1834_1958['datetime'].dt.month > 7))
# diurnal_temp_range = df_irtemp_1834_1958['max c'] - df_irtemp_1834_1958['min c']
# ice_days = (df_irtemp_1834_1958['max c'] < 0) # df.groupby('Year')['TX'].apply(lambda x: (x < 0).sum())
# coldest_night = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['datetime'].dt.month)['min c'].min() == df_irtemp_1834_1958['min c']) # df.groupby(['Year', 'Month'])['TN'].min()
# hottest_night = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['datetime'].dt.month)['min c'].max() == df_irtemp_1834_1958['min c']) # df.groupby(['Year', 'Month'])['TN'].max()
# hottest_day = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['datetime'].dt.month)['max c'].max() == df_irtemp_1834_1958['max c']) # df.groupby(['Year', 'Month'])['TX'].max()
# coldest_day = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['datetime'].dt.month)['max c'].min() == df_irtemp_1834_1958['max c']) # df.groupby(['Year', 'Month'])['TX'].min()
# cool_days = (df_irtemp_1834_1958['max c'] < np.percentile(df_irtemp_1834_1958['max c'], 10))
# cool_nights = (df_irtemp_1834_1958['min c'] < np.percentile(df_irtemp_1834_1958['min c'], 10))
# warm_nights = (df_irtemp_1834_1958['min c'] > np.percentile(df_irtemp_1834_1958['min c'], 90))
# warm_days = (df_irtemp_1834_1958['max c'] > np.percentile(df_irtemp_1834_1958['max c'], 90))

# df_irtemp_1834_1958['cold_Spell_Duration'] = np.where(cold_spell, 1, 0)
# df_irtemp_1834_1958['Warm_Spell_Duration'] = np.where(warm_spell, 1, 0)
# df_irtemp_1834_1958['Frost_Days'] = np.where(frost_days, 1, 0)
# df_irtemp_1834_1958['Growseas_Length'] = np.where(growseas_length, 1, 0)
# df_irtemp_1834_1958['Diurnal_Temp_Range'] = diurnal_temp_range
# df_irtemp_1834_1958['Ice_Days'] = np.where(ice_days, 1, 0)
# df_irtemp_1834_1958['Coldest_Night'] = np.where(coldest_night, df_irtemp_1834_1958['min c'], np.nan)
# df_irtemp_1834_1958['Hottest_Night'] = np.where(hottest_night, df_irtemp_1834_1958['min c'], np.nan)
# df_irtemp_1834_1958['Hottest_Day'] = np.where(hottest_day, df_irtemp_1834_1958['max c'], np.nan)
# df_irtemp_1834_1958['Coldest_Day'] = np.where(coldest_day, df_irtemp_1834_1958['max c'], np.nan)
# df_irtemp_1834_1958['Cool_Days_Percentage'] = (df_irtemp_1834_1958['max c'] < np.percentile(df_irtemp_1834_1958['max c'], 10)).astype(int)
# df_irtemp_1834_1958['Cool_Nights_Percentage'] = (df_irtemp_1834_1958['min c'] < np.percentile(df_irtemp_1834_1958['min c'], 10)).astype(int)
# df_irtemp_1834_1958['Warm_Nights_Percentage'] = (df_irtemp_1834_1958['min c'] > np.percentile(df_irtemp_1834_1958['min c'], 90)).astype(int)
# df_irtemp_1834_1958['Warm_Days_Percentage'] = (df_irtemp_1834_1958['max c'] > np.percentile(df_irtemp_1834_1958['max c'], 90)).astype(int)

In [58]:
# ds = df.to_xarray()
# ds.set_coords('time').swap_dims({'index': 'time'})
# ds.to_netcdf('sampledata.1991-2010.nc')
# bp = [datetime.datetime(1850, 1, 1), datetime.datetime(1900, 12, 31)] # base period
# tr = [datetime.datetime(1834, 1, 1), datetime.datetime(2023, 10, 31)] # time range
# file = 'sampledata.1991-2010.nc' 
# res = icclim.indices(
#     index_group="all",
#     in_files=file,
#     base_period_time_range=bp,
#     time_range=tr,
#     out_file="heat_indices.nc",
# )

# file = "sampledata.1991-2010.nc"
# res = icclim.indices(
#     index_group=IndexGroup.HEAT,
#     in_files=file,
#     base_period_time_range=bp,
#     time_range=tr,
#     out_file="heat_indices.nc",
# )


#### 2.5.2.5 Ireland's ETCCDI Temperature Indices Evaluation

Ireland's Expert Team on Climate Change Detection and Indices (ETCCDI) evaluation was done according to Murphy et al. (2023) as described in Table 1 on Glasnevin dataset, with the exception that the standard climate normal period was from 1850 to 1900. Icclim Python package was imported (icclim.readthedocs.io, n.d.) and indices were computed according to icclim.readthedocs.io ecad (n.d.).

##### Table 1: Expert Team on Climate Change Detection and Indices (ETCCDI) Temperature Indices (Murphy et al., 2023)

| **Label**| **Definition** |
| :-------: | :------: |
| Cold spell duration (CSDI)| Annual count of days with at least 6 consecutive days when daily minimum air temperature (TN) < 10th percentile of SCN (standard climate normal period 1850–1900) |
| Warm spell duration (WSDI)| Annual count of days with at least 6 consecutive days when daily maximum air temperature (TX) > 90th percentile of SCN (standard climate normal period 1850–1900) |
| Frost days (FD) | Annual count when daily minimum air temperature (TN) < 0°C |
| Growing season length (GSL) | Annual count between the first span of at least 6 days with TM > 5 ◦ C and the first span after July 1st of 6 days with daily mean air temperature (TM) < 5°C |
| Diurnal temp range (DTR) | Monthly mean difference between (TX) and daily minimum air temperature (TN) |
| Ice days (ID)| Annual count daily maximum air temperature (TX) < 0°C|
| Coldest night (TNn) | Monthly minimum value of daily minimum air temperature (TN) |
| Hottest night (TNx)|  Monthly maximum value of daily minimum air temperature (TN) |
| Hottest day (TXx)| Monthly maximum value of daily maximum air temperature (TX) |
| Coldest day (TXn) | Monthly minimum value of daily maximum air temperature (TX) |
| Cool days (TX10p) | % of days when daily maximum air temperature (TX) < 10th percentile of SCN (standard climate normal period 1850–1900) |
| Cool nights (TN10p) | % of days when daily minimum air temperature (TN) < 10th percentile of SCN (standard climate normal period 1850–1900) |
| Warm nights (TN90p) | % of days when daily minimum air temperature (TN) > 90th percentile of SCN (standard climate normal period 1850–1900) |
| Warm days (TX90p) |  % of days where daily maximum air temperature (TX) > 90th percentile of SCN (standard climate normal period 1850–1900) |

The seasons were defined as winter (December, January,
February), spring (March, April, May), summer (June, July, August) and
autumn (September, October, November)

https://icclim.readthedocs.io/en/latest/how_to/recipes_ecad.html

https://github.com/AusClimateService/indices

In [59]:
# # Cold spell duration = Annual count of days with at least 6 consecutive days when daily minimum air temperature (TN) < 10th percentile of SCN (standard climate normal period between 1850–1900), 
# # Warm spell duration = Annual count of days with at least 6 consecutive days when daily maximum air temperature (TX) > 90th percentile of SCN (standard climate normal period between 1850–1900), 
# # Frost days = Annual count when daily minimum air temperature (TN) < 0 Celcius, 
# # Growseas length = Annual count between the first span of at least 6 days with TM > 5 Celcius and the first span after July 1st of 6 days with daily mean air temperature (TM) < 5 Celcius, 
# # Diurnal temp range = Monthly mean difference between (TX) and daily minimum air temperature (TN), 
# # Ice days = Annual count daily maximum air temperature (TX) < 0 Celcius, 
# # Coldest night = Monthly minimum value of daily minimum air temperature (TN), 
# # Hottest night = Monthly maximum value of daily minimum air temperature (TN), 
# # Hottest day = Monthly maximum value of daily maximum air temperature (TX), 
# # Coldest day = Monthly minimum value of daily maximum air temperature (TX), 
# # Cool days = % of days when daily maximum air temperature (TX) < 10th percentile of SCN (standard climate normal period between 1850–1900), 
# # Cool nights = % of days when daily minimum air temperature (TN) < 10th percentile of SCN (standard climate normal period between 1850–1900), 
# # Warm nights = % of days when daily minimum air temperature (TN) > 90th percentile of SCN (standard climate normal period between 1850–1900), 
# # Warm days = % of days where daily maximum air temperature (TX) > 90th percentile of SCN (standard climate normal period between 1850–1900).

# cold_spell = (df_irtemp_1834_1958['min c'].rolling(window=6).min() < np.percentile(df_irtemp_1834_1958['min c'], 10))
# warm_spell = (df_irtemp_1834_1958['max c'].rolling(window=6).max() > np.percentile(df_irtemp_1834_1958['max c'], 90))
# frost_days = (df_irtemp_1834_1958['min c'] < 0)
# growseas_length = ((df_irtemp_1834_1958['irtemp c'] > 5) & (df_irtemp_1834_1958['dmy'].dt.month < 7)) | ((df_irtemp_1834_1958['irtemp c'] < 5) & (df_irtemp_1834_1958['dmy'].dt.month > 7))
# diurnal_temp_range = df_irtemp_1834_1958['max c'] - df_irtemp_1834_1958['min c']
# ice_days = (df_irtemp_1834_1958['max c'] < 0) # df.groupby('Year')['TX'].apply(lambda x: (x < 0).sum())
# coldest_night = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['dmy'].dt.month)['min c'].min() == df_irtemp_1834_1958['min c']) # df.groupby(['Year', 'Month'])['TN'].min()
# hottest_night = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['dmy'].dt.month)['min c'].max() == df_irtemp_1834_1958['min c']) # df.groupby(['Year', 'Month'])['TN'].max()
# hottest_day = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['dmy'].dt.month)['max c'].max() == df_irtemp_1834_1958['max c']) # df.groupby(['Year', 'Month'])['TX'].max()
# coldest_day = (df_irtemp_1834_1958.groupby(df_irtemp_1834_1958['dmy'].dt.month)['max c'].min() == df_irtemp_1834_1958['max c']) # df.groupby(['Year', 'Month'])['TX'].min()
# cool_days = (df_irtemp_1834_1958['max c'] < np.percentile(df_irtemp_1834_1958['max c'], 10))
# cool_nights = (df_irtemp_1834_1958['min c'] < np.percentile(df_irtemp_1834_1958['min c'], 10))
# warm_nights = (df_irtemp_1834_1958['min c'] > np.percentile(df_irtemp_1834_1958['min c'], 90))
# warm_days = (df_irtemp_1834_1958['max c'] > np.percentile(df_irtemp_1834_1958['max c'], 90))

# df_irtemp_1834_1958['cold_Spell_Duration'] = np.where(cold_spell, 1, 0)
# df_irtemp_1834_1958['Warm_Spell_Duration'] = np.where(warm_spell, 1, 0)
# df_irtemp_1834_1958['Frost_Days'] = np.where(frost_days, 1, 0)
# df_irtemp_1834_1958['Growseas_Length'] = np.where(growseas_length, 1, 0)
# df_irtemp_1834_1958['Diurnal_Temp_Range'] = diurnal_temp_range
# df_irtemp_1834_1958['Ice_Days'] = np.where(ice_days, 1, 0)
# df_irtemp_1834_1958['Coldest_Night'] = np.where(coldest_night, df_irtemp_1834_1958['min c'], np.nan)
# df_irtemp_1834_1958['Hottest_Night'] = np.where(hottest_night, df_irtemp_1834_1958['min c'], np.nan)
# df_irtemp_1834_1958['Hottest_Day'] = np.where(hottest_day, df_irtemp_1834_1958['max c'], np.nan)
# df_irtemp_1834_1958['Coldest_Day'] = np.where(coldest_day, df_irtemp_1834_1958['max c'], np.nan)
# df_irtemp_1834_1958['Cool_Days_Percentage'] = (df_irtemp_1834_1958['max c'] < np.percentile(df_irtemp_1834_1958['max c'], 10)).astype(int)
# df_irtemp_1834_1958['Cool_Nights_Percentage'] = (df_irtemp_1834_1958['min c'] < np.percentile(df_irtemp_1834_1958['min c'], 10)).astype(int)
# df_irtemp_1834_1958['Warm_Nights_Percentage'] = (df_irtemp_1834_1958['min c'] > np.percentile(df_irtemp_1834_1958['min c'], 90)).astype(int)
# df_irtemp_1834_1958['Warm_Days_Percentage'] = (df_irtemp_1834_1958['max c'] > np.percentile(df_irtemp_1834_1958['max c'], 90)).astype(int)

In [60]:
df_all = pd.read_csv('df_all.csv')
df_all.head(3)

,year reset,irrain mean,irtemp mean,co2 ppmv pre2001,co2 ppmv luthi,co2 ppmv post2001,co2 mean,temp c ffill,temp c mean ffill,anom c,modern temp c,modern anom c,anom modern anom,ch4 ppbv pre1937,ch4 ppbv 1938,ch4 ppbv 1997,ch4 mean
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-61.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,999.0,NaN,NaN,NaN,NaN,NaN,NaN,-61.75,-61.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1069.0,NaN,NaN,NaN,NaN,NaN,NaN,-61.75,-61.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_alin = df_all[
                        (df_all['year reset'] >= 821719.08333) &
                         (df_all['year reset'] <= 821911.9166667)
                         ][[
                             'year reset', 
                             'irtemp mean', 
                             'anom modern anom'
                             ]] .copy()
df_alin

,year reset,irtemp mean,anom modern anom
18582,821719.083333,NaN,NaN
18583,821719.083333,NaN,NaN
18584,821719.083333,NaN,NaN
18585,821719.083333,NaN,NaN
18586,821719.083333,NaN,NaN
...,...,...,...
68983,821911.583333,14.9,NaN
68984,821911.666667,15.4,NaN
68985,821911.750000,14.5,NaN
68986,821911.833333,11.3,NaN


In [62]:
#df_alin['year reset'] = pd.to_datetime(df_alin['year reset'])

In [63]:
#df_alin.set_index('year reset', drop=True, inplace=True)

In [64]:
it, frac = 41, 0.25

# the data has to be yearly to use it=41
irish_lowess = lowess(endog=df_alin['irtemp mean'], 
            exog=df_alin.index,
            it=it,
            frac=frac
            )

# modern anom is yearly, so can use it=41

global_lowess = lowess(endog=df_alin['anom modern anom'],
                exog=df_alin.index,
                it=it,
                frac=frac
                )

irish_lowess = pd.DataFrame(irish_lowess)
irish_lowess.columns = ['date', 'irish smoothed']

global_lowess = pd.DataFrame(global_lowess)
global_lowess.columns = ['date', 'global smoothed']

In [65]:
df_alin = df_alin.merge(irish_lowess, 
                       left_index=True, right_on='date',
                       how='left')
df_alin

,year reset,irtemp mean,anom modern anom,date,irish smoothed
NaN,821719.083333,NaN,NaN,18582.0,NaN
NaN,821719.083333,NaN,NaN,18583.0,NaN
NaN,821719.083333,NaN,NaN,18584.0,NaN
NaN,821719.083333,NaN,NaN,18585.0,NaN
NaN,821719.083333,NaN,NaN,18586.0,NaN
...,...,...,...,...,...
50131.0,821911.583333,14.9,NaN,68983.0,11.507476
50132.0,821911.666667,15.4,NaN,68984.0,11.507226
50133.0,821911.750000,14.5,NaN,68985.0,11.506975
50134.0,821911.833333,11.3,NaN,68986.0,11.506725


In [66]:
df_alin.merge(global_lowess, left_on='date', right_on='date', how='left').to_csv('df_alin.csv', index=True)

In [68]:
df_alin = pd.read_csv('df_alin.csv')
df_alin.head(3)

,Unnamed: 0,year reset,irtemp mean,anom modern anom,date,irish smoothed,global smoothed
0,0,821719.083333,NaN,NaN,18582.0,NaN,NaN
1,1,821719.083333,NaN,NaN,18583.0,NaN,NaN
2,2,821719.083333,NaN,NaN,18584.0,NaN,NaN
